CellPose tutorial
https://github.com/MouseLand/cellpose/blob/main/notebooks/run_cellpose_GPU.ipynb

In [1]:
! nvidia-smi

Sun Jun 25 14:31:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 495.46       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:00:07.0 Off |                  N/A |
| 20%   26C    P8     9W / 250W |      2MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:00:0C.0 Off |                  N/A |
| 28%   

In [1]:
import napari
from dask_image.imread import imread

In [20]:
# Set up root directory
root = "/scratch/iss_decoding/workshop_data/segmentation/in/"
out_dir = "./out"

In [3]:
! mkdir "{out_dir}"

mkdir: cannot create directory ‘./out’: File exists


In [4]:
ref_DAPI = imread(f"{root}/demo_optflow_seg_optflow_reg_result_stack.tif")[0].squeeze()
ref_DAPI.shape

(4114, 4369)

In [5]:
viewer = napari.Viewer()

In [6]:
viewer.add_image(ref_DAPI)

<Image layer 'ref_DAPI' at 0x7fbbd85d9760>

In [7]:
from cellpose import models, core
import tifffile as tf
import matplotlib.pyplot as plt

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

>>> GPU activated? 1


In [12]:
# model = models.Cellpose(gpu=use_GPU, model_type='cyto2') # Initialize model
model = models.Cellpose(gpu=False, model_type='cyto2') # Initialize model, if GPU has issues

In [13]:
channels = [[0, 0]] # Use just DAPI channel for segmentation

In [14]:
ref_DAPI.shape

(4114, 4369)

In [15]:
masks, flows, styles, diams = model.eval(ref_DAPI,
                                         diameter=60,
                                         flow_threshold=None,
                                         channels=channels) # Run segmentation

In [16]:
viewer.add_image(masks)
viewer.add_labels(masks)

<Labels layer 'masks [1]' at 0x7fbc7789a3a0>

In [17]:
from skimage.segmentation import expand_labels

In [18]:
expanded_masks = expand_labels(masks, distance=10)

In [19]:
viewer.add_labels(expanded_masks)

<Labels layer 'expanded_masks' at 0x7fbc45bed250>

In [22]:
tf.imwrite(f"{out_dir}/cellpose_segmentation.tif", expanded_masks)